train and valid datasets are from [2014, 2022], split depending on `train_valid_ratio`.

test is from [2022, 2024].

The dataset if from 2014 to 2024. It is IMPORTANT to set test_end = 2025! Otherwise the test dataset is cut short. See `ModelFeatures::__init__()`.

ModelFeatures has scalers.

**TODO** `ModelFeatures.transform_inputs()`

**TODO** Train a model and understand how this works end-to-end!!!

In [3]:
import pandas as pd
from mom_trans.model_inputs import ModelFeatures
from settings.default import QUANDL_TICKERS

ASSET_CLASS_MAPPING = dict(zip(QUANDL_TICKERS, ["COMB"] * len(QUANDL_TICKERS)))

params = {
    'architecture': 'LSTM', 
    'total_time_steps': 63, 
    'early_stopping_patience': 25, 
    'multiprocessing_workers': 32, 
    'num_epochs': 300, 
    'fill_blank_dates': False, 
    'split_tickers_individually': True, 
    'random_search_iterations': 50, 
    'evaluate_diversified_val_sharpe': True, 
    'train_valid_ratio': 0.9, 
    'time_features': False, 
    'force_output_sharpe_length': None
}

train_interval = (2014, 2022, 2025)
changepoint_lbws = None # depends on selected architecture
asset_class_dictionary = ASSET_CLASS_MAPPING

features_file_path = "data/quandl_cpd_nonelbw.csv"
raw_data = pd.read_csv(features_file_path, index_col=0, parse_dates=True)
raw_data["date"] = raw_data["date"].astype("datetime64[ns]")

model_features = ModelFeatures(
    raw_data,
    params["total_time_steps"],
    start_boundary=train_interval[0],
    test_boundary=train_interval[1],
    test_end=train_interval[2],
    changepoint_lbws=changepoint_lbws,
    split_tickers_individually=params["split_tickers_individually"],
    train_valid_ratio=params["train_valid_ratio"],
    add_ticker_as_static=(params["architecture"] == "TFT"),
    time_features=params["time_features"],
    lags=params["force_output_sharpe_length"],
    asset_class_dictionary=asset_class_dictionary,
)

# model_features.valid.keys()
# dict_keys(['active_entries', 'outputs', 'identifier', 'date', 'inputs'])

# model_features.train.keys()
# dict_keys(['active_entries', 'outputs', 'identifier', 'date', 'inputs'])

print(model_features.train['inputs'].shape)
print(model_features.valid['inputs'].shape)
print(model_features.test_fixed['inputs'].shape)
print(model_features.test_sliding['inputs'].shape)

print(model_features.train['outputs'])

(435, 63, 8)
(61, 63, 8)
(192, 63, 8)
(11488, 63, 8)
[[[-0.01041057]
  [-0.00526621]
  [ 0.01332012]
  ...
  [ 0.02034665]
  [ 0.00282319]
  [-0.0088297 ]]

 [[-0.02959624]
  [-0.00930351]
  [ 0.00907429]
  ...
  [ 0.01534445]
  [ 0.00231976]
  [ 0.00883062]]

 [[-0.00427861]
  [-0.01715691]
  [ 0.01005884]
  ...
  [ 0.00180748]
  [-0.02070694]
  [ 0.        ]]

 ...

 [[ 0.0047009 ]
  [-0.00050913]
  [ 0.00542171]
  ...
  [ 0.00893885]
  [-0.00290986]
  [ 0.00921424]]

 [[-0.01295914]
  [ 0.01568048]
  [ 0.00440375]
  ...
  [ 0.00053885]
  [-0.00827395]
  [ 0.00729724]]

 [[ 0.01007798]
  [ 0.01908827]
  [ 0.01965417]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]]
